In [77]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import plotly.express as px
from pathlib import Path
import holidays

from pathlib import Path
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import Huber
from sklearn.preprocessing import MinMaxScaler



In [69]:
def upload_data():
    data_path = str(Path.cwd().parent) + "\\Data\\EPC\\Power Consumption Data.csv"

    df = pd.read_csv(data_path)

    df = df[df["real_consumption"] > 0]
    df = df[df['real_consumption'] <= df['real_consumption'].mean() + 4 * df['real_consumption'].std()]

    df['time'] = pd.to_datetime(df['time'])
    df = df.sort_values(by='time',ascending=True)

    return df

def data_metrics(data, real, predicted):

    y_true = data[real]
    y_pred = data[predicted]


    def r2_score_tf(y_true, y_pred):
        """
        Computes the R² (coefficient of determination) score in TensorFlow.
        
        Parameters:
        y_true : Tensor
            Ground truth values.
        y_pred : Tensor
            Predicted values.
    
        Returns:
        Tensor
            The R² score.
        """
        
        
        y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    
        unexplained_error = tf.reduce_sum(tf.square(y_true - y_pred))  # SSE
        total_error = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))  # SST
    
        r_squared = 1.0 - (unexplained_error / (total_error + tf.keras.backend.epsilon()))  # Prevent division by zero
        return r_squared


    # Calculate metrics
    mae = tf.keras.losses.MeanAbsoluteError()(y_true, y_pred).numpy()
    mse = tf.keras.losses.MeanSquaredError()(y_true, y_pred).numpy()
    rmse = np.sqrt(mse)
    r2 = r2_score_tf(y_true, y_pred)


    # MAE (Mean Absolute Error):
    # Lower values are better; good MAE depends on the scale of 'real_consumption'.
    # As a rule of thumb, MAE should be significantly smaller than the mean of the target variable.
    # Lower is better. Ideally, MAE should be much less than the average value of y_true.
    print(f"MAE: {mae:.4f}")

    # MSE (Mean Squared Error):
    # Similar to MAE but penalizes large errors more heavily. A smaller MSE is better.
    # Compare MSE to the variance of 'real_consumption' for context.
    # Lower is better. MSE should ideally be close to zero relative to the variance of y_true.
    print(f"MSE: {mse:.4f}")



    # RMSE (Root Mean Squared Error):
    # RMSE is the square root of MSE and is in the same units as 'real_consumption'.
    # A good RMSE is often close to the standard deviation of 'real_consumption'.
    # Lower is better. RMSE should be comparable to or less than the standard deviation of y_true."
    print(f"RMSE: {rmse:.4f}")



    # R² (Coefficient of Determination):
    # R² measures how well the predictions explain the variability of the data.
    # Values close to 1.0 are excellent, indicating the model explains most of the variance.
    # Negative values indicate poor fit.
    # Closer to 1.0 is better. Values > 0.7 are generally good; < 0.5 indicates underfitting.
    print(f"R²: {r2:.4f}")

def feature_engineering(data):

    # Extracting basic time-based features
    data['hour'] = data['time'].dt.hour  # Hour of the day
    data['minute'] = data['time'].dt.minute  # Minute
    data['day_of_week'] = data['time'].dt.dayofweek  # Day of the week (0=Monday, 6=Sunday)
    data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)  # Weekend flag
    data['day_of_month'] = data['time'].dt.day
    data['week_of_year'] = data['time'].dt.isocalendar().week
    data['month'] = data['time'].dt.month
    data['quarter'] = data['time'].dt.quarter
    data['year'] = data['time'].dt.year


    # Generate lag features for temporal dependency modeling
    for lag in range(1, 5):  # Create lag features for the past 4 time steps
        data[f'lag_{lag}'] = data['real_consumption'].shift(lag)


    # Generate exponential moving averages
    for span in [3, 5]:  # Spans of size 3, 5, and 7
        data[f'ema_{span}'] = data['real_consumption'].ewm(span=span, adjust=False).mean()

    # Rolling average over a longer period (e.g., weekly and monthly moving averages)
    data['weekly_avg'] = data['real_consumption'].rolling(window=7*24*20, min_periods=1).mean()  # Weekly moving avg
    data['monthly_avg'] = data['real_consumption'].rolling(window=30*24*20, min_periods=1).mean()  # Monthly moving avg


    # Percentage change in real consumption
    data['pct_change'] = data['real_consumption'].pct_change()

    data['hour_sin'] = np.sin(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (sine)
    data['hour_cos'] = np.cos(2 * np.pi * data['hour'] / 24)  # Cyclic hour feature (cosine)

    data['day_of_week_sin'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
    data['day_of_week_cos'] = np.cos(2 * np.pi * data['day_of_week'] / 7)

    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

    data['week_of_year_sin'] = np.sin(2 * np.pi * data['week_of_year'] / 52)
    data['week_of_year_cos'] = np.cos(2 * np.pi * data['week_of_year'] / 52)




    # Get Georgia holidays for all years in the dataset
    georgia_holidays = holidays.Georgia(years=range(data["year"].min(), data["year"].max() + 1))

    data["date"] = data["time"].dt.date

    # Create holiday feature (1 if it's a holiday, 0 otherwise)
    data['is_holiday'] = data["date"].map(lambda x: 1 if x in georgia_holidays else 0)

    # Add features for the day before and after a holiday
    data['is_day_before_holiday'] = data["date"].map(lambda x: 1 if (x - pd.Timedelta(days=1)) in georgia_holidays else 0)
    data['is_day_after_holiday'] = data["date"].map(lambda x: 1 if (x + pd.Timedelta(days=1)) in georgia_holidays else 0)



    return  data



In [70]:
df = upload_data()

data_metrics(data=df, real="real_consumption", predicted="predicted_consumption")

df = feature_engineering(df)


MAE: 56.7256
MSE: 5824.3340
RMSE: 76.3173
R²: 0.9176


In [91]:
df

,time,real_consumption,predicted_consumption,hour,minute,day_of_week,is_weekend,day_of_month,week_of_year,month,...,day_of_week_sin,day_of_week_cos,month_sin,month_cos,week_of_year_sin,week_of_year_cos,date,is_holiday,is_day_before_holiday,is_day_after_holiday
time,,,,,,,,,,,,,,,,,,,,,
2020-10-14 18:39:00,2020-10-14 18:39:00,1437.405518,1530.0,18,39,2,0,14,42,10,...,0.974928,-0.222521,-0.866025,0.500000,-0.935016,0.354605,2020-10-14,1,0,0
2020-10-14 18:42:00,2020-10-14 18:42:00,1439.720825,1530.0,18,42,2,0,14,42,10,...,0.974928,-0.222521,-0.866025,0.500000,-0.935016,0.354605,2020-10-14,1,0,0
2020-10-14 18:45:00,2020-10-14 18:45:00,1449.654541,1530.0,18,45,2,0,14,42,10,...,0.974928,-0.222521,-0.866025,0.500000,-0.935016,0.354605,2020-10-14,1,0,0
2020-10-14 18:48:00,2020-10-14 18:48:00,1468.833984,1530.0,18,48,2,0,14,42,10,...,0.974928,-0.222521,-0.866025,0.500000,-0.935016,0.354605,2020-10-14,1,0,0
2020-10-14 18:51:00,2020-10-14 18:51:00,1491.803955,1530.0,18,51,2,0,14,42,10,...,0.974928,-0.222521,-0.866025,0.500000,-0.935016,0.354605,2020-10-14,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22 18:42:00,2024-11-22 18:42:00,1942.344971,1940.0,18,42,4,0,22,47,11,...,-0.433884,-0.900969,-0.500000,0.866025,-0.568065,0.822984,2024-11-22,0,0,1
2024-11-22 18:45:00,2024-11-22 18:45:00,1922.017700,1940.0,18,45,4,0,22,47,11,...,-0.433884,-0.900969,-0.500000,0.866025,-0.568065,0.822984,2024-11-22,0,0,1
2024-11-22 18:48:00,2024-11-22 18:48:00,1936.747803,1940.0,18,48,4,0,22,47,11,...,-0.433884,-0.900969,-0.500000,0.866025,-0.568065,0.822984,2024-11-22,0,0,1


In [94]:
df.index = pd.to_datetime(df['time'], format='%d.%m.%Y %H:%M:%S')
# temp = df[['real_consumption',"hour","weekly_avg","monthly_avg","hour_sin",
# "hour_cos","lag_1","ema_3","month_sin","month_cos","week_of_year_sin","week_of_year_cos"]]

temp = df[['real_consumption',"hour","weekly_avg","monthly_avg","ema_3","month","week_of_year","day_of_month","is_weekend","day_of_week"]]

scaler = MinMaxScaler()
temp = pd.DataFrame(scaler.fit_transform(temp), columns=temp.columns)

In [95]:
# [[[1], [2], [3], [4], [5]]] [6]
# [[[2], [3], [4], [5], [6]]] [7]
# [[[3], [4], [5], [6], [7]]] [8]

def df_to_X_y(df, window_size=5):
    df_as_np = df.to_numpy().astype("float32")  # Convert to float32
    X, y = [], []

    for i in range(len(df_as_np)-window_size):
        row = df_as_np[i:i+window_size]  # Past `window_size` rows as input
        X.append(row)
        label = df_as_np[i+window_size, 0]  # Target: `real_consumption`
        y.append(label)

    return np.array(X, dtype="float32"), np.array(y, dtype="float32")  # Convert to float32



In [96]:
WINDOW_SIZE = 3*20
X1, y1 = df_to_X_y(temp, WINDOW_SIZE)
X1.shape, y1.shape

((708221, 120, 10), (708221,))

In [97]:
X_train1, y_train1 = X1[:600000], y1[:600000]
X_val1, y_val1 = X1[600000:650000], y1[600000:650000]
X_test1, y_test1 = X1[650000:], y1[650000:]
X_train1.shape, y_train1.shape, X_val1.shape, y_val1.shape, X_test1.shape, y_test1.shape

((600000, 120, 10),
 (600000,),
 (50000, 120, 10),
 (50000,),
 (58221, 120, 10),
 (58221,))

In [98]:
model = Sequential([
    InputLayer((X1.shape[1], X1.shape[2])),  # Updated input shape
    LSTM(64, return_sequences=True),  # First LSTM layer
    LSTM(32),  # Second LSTM layer
    Dense(8, activation='relu'),
    Dense(1, activation='linear')  # Predicting `real_consumption`
])


model.compile(
    loss=Huber(delta=1.0),
    optimizer=Adam(learning_rate=0.0001),
    metrics=[RootMeanSquaredError()]
)


BATCH_SIZE = 256
EPOCHS = 50

train_dataset = tf.data.Dataset.from_tensor_slices((X_train1, y_train1)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_val1, y_val1)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Train with ModelCheckpoint & EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = str(Path.cwd().parent / "Data" / "best_model.keras")

cp1 = ModelCheckpoint(filepath=checkpoint_path, save_best_only=True, monitor="val_loss", verbose=1)

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=1)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=EPOCHS,
    callbacks=[cp1, early_stopping]
)


Epoch 1/50
 180/2344 ━━━━━━━━━━━━━━━━━━━━ 6:48 189ms/step - loss: 0.0304 - root_mean_squared_error: 0.2452

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model(checkpoint_path)

train_predictions = model1.predict(X_train1).flatten()
train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train1})

val_predictions = model1.predict(X_val1).flatten()
val_results = pd.DataFrame(data={'Val Predictions':val_predictions, 'Actuals':y_val1})

test_predictions = model1.predict(X_test1).flatten()
test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test1})


In [ ]:
test_results["index"] = temp[650005:].index
test_results

In [ ]:
temp[650005:650050]

In [ ]:
temp[650010:650011].copy()

In [ ]:
predict_points = 100
test_predictions = []
last_several_month = temp[650005:650011].copy()


current_batch = df_to_X_y(last_several_month, window_size=5)[0]

for i in range(predict_points):

    # get the prediction value for the first batch
    current_pred = model1.predict(current_batch).flatten()

    # append the prediction into the array
    test_predictions.append(current_pred)

    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)



results = temp[650010:(650010 + predict_points)].copy().reset_index()
results["predic"] = pd.DataFrame(test_predictions).rename(columns={0:"predict"})["predict"]
results

In [ ]:
# Create an interactive plot using Plotly
fig = px.line(results, x='time', y=['real_consumption', 'predic'], title="Interactive Time Series Plot")

# Customize the plot
fig.update_layout(
    xaxis_title="Timestamp",
    yaxis_title="Values",
    legend_title="Legend",
    xaxis_rangeslider_visible=True  # Enables the date slicer
)

# Show the plot
fig.show()

In [ ]:
data_metrics(data=test_results, real="Actuals", predicted="Test Predictions")